# Document Clustering

This notebook demonstrates a relatively simple approach to automated record clustering based on keywords contents.

The purpose of the notebook is to:
 * demonstrate use of record management APIs
 * show that it is practical to use machine learning algorithms to automatically discover document clusters
 * provide refernce implementation that could be enhanced
 * generate a model that can subsequently be used to demonstration of automatic record classification

In a production system, unsupervised clustering techniques such as this would probably be used in an ensemble with ontology-based classification as well as supervised learning of labels that have been applied by human record mangers to a reference set.

In [1]:
# Import all the packages to be used
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd

### Fetching data from the API

The first step in this recipie is to fetch data from the search API (due to a small current bug, we are actually using the raw elasticsearch API behind the Platform Search API.

This code is wrapped in a class with two main methods -- `get_total()` and `get_records()`.

In [2]:
import requests

class SearchInterface(object):
    
    def __init__(self):
        self.base_url = 'https://digitalrecords.showthething.com/api/search/v0/records/'
        self.get_total()
        
    def get_total(self):
        """ Gets the total number of records that match the query """
        resp = self._fetch_batch(0, limit=1)
        data = resp.json()
        self.total = data['hits']['count']
        
    def _fetch_batch(self, offset, limit=1000):
        # The `exists`: `keywords` params makes sure that only those records with
        # keywords are fetched from the database
        params = {'limit': limit, 'offset': offset, 'exists': 'keywords'}
        resp = requests.get(self.base_url, params=params)
        return resp
    
    def get_records(self, test_run=False):
        print('Total matches: %s' % self.total)
        if test_run:
            # We just limit to the 10,000 records for the test run
            records_num = 10000
            print('Total records to fetch: %s (TEST RUN)' % records_num)
        else:
            records_num = self.total
            print('Total records to fetch: %s' % records_num)
        results = []
        batches = range(0, self.total, 1000)
        print('\nNow fetching records from ES...')
        for batch_offset in batches:
            fetch_count = batch_offset + 1000
            if fetch_count <= records_num:
                resp = self._fetch_batch(batch_offset)
                if resp.status_code == 200:
                    data = resp.json()
                    results += data['hits']['results']
                else:
                    raise Exception('Error fetching records from the server!')
                print(' - Fetched %s records out of %s' % (fetch_count, records_num))
            else:
                break
        print('Finished fetching records from ES\n')
        return results

This class can now be used to access metadata from the API. This is currently the slowest part of the process, even with the `test_run=True` parameter limiting it to 50,000 records. The approach of pulling keyword metadata down to a script works for hundreds of thousands of millions of records, but becomes impractical for tens or hundreds of millions of records. Future implementations should use a "send the code to the data (not the data to the code)" type of approach, such as map-reduce. 

In [3]:
# Fetch the records in test_run mode, which means it will only get 10,000 matching records
search = SearchInterface()
records = search.get_records(test_run=True)

Total matches: 16899588
Total records to fetch: 10000 (TEST RUN)

Now fetching records from ES...
 - Fetched 1000 records out of 10000
 - Fetched 2000 records out of 10000
 - Fetched 3000 records out of 10000
 - Fetched 4000 records out of 10000
 - Fetched 5000 records out of 10000
 - Fetched 6000 records out of 10000
 - Fetched 7000 records out of 10000
 - Fetched 8000 records out of 10000
 - Fetched 9000 records out of 10000
 - Fetched 10000 records out of 10000
Finished fetching records from ES



As usual, the data needs to be cleaned up a little bit before it can be used.

In [4]:
import pandas as pd

# Build a dataframe out of the records, taking only the uuid and keywords per record
df = pd.DataFrame(records, columns=['uuid', 'keywords'])

# Let's just rename the `uuid` column to `id`
df.rename(columns={'uuid': 'id'}, inplace=True)

# Replace NaN's with empty string
df.fillna('', inplace=True)

# Make sure the keywords column is of string data type
df['keywords'] = df['keywords'].astype(str)

# Show a sample of the dataframe
df.sample(10)

,id,keywords
8466,6ab02346-907b-44a5-be92-ed4da2d08ea0,"['Pathway', 'experience', 'Flatwater', 'Traini..."
4571,f8cca64b-7946-4efc-b60b-88e830af84c1,"['Turbines', 'County', 'Independent', 'Natural..."
6553,81911597-1dff-4b26-83c8-ba3dd90e8a5e,"['followers', 'Twitter?', 'Twitter']"
4696,158cb0ba-a443-4971-b583-d73ddc7c0194,"['Lutheran', 'Welcome', 'Upcoming', 'Descripti..."
4637,44d701a6-d786-4bd7-8f2b-b744e119dd0e,"['Independent', 'renewable', 'Elegant', 'Minis..."
7158,c345f07a-31a2-461d-8611-6fb67908e08e,"['Description:', 'Reviews', 'Singlet', 'Newsle..."
4690,c0bcce72-47a9-432a-a263-eb31004c2575,"['Service', 'Lutheran', 'Welcome', 'Illawarra'..."
13,13da464a-11f3-472e-8945-d6d1af96c3dc,['uâ']
6580,31c2c52c-9a43-438e-8c47-275539431f78,"['Arboretum', 'euroaarboretum', 'responses']"
1715,c632344a-9a6b-4979-96d4-0e751fe0823f,"['19/11/2016', 'ARTICLESMORE', 'COMMING', 'INV..."


Nowv we need to create a tokenize method to extract individual words (remove punctuation, stopwords, etc).

The `tokenize()` function that we create below is used by built-in vectorization classes from the SciKit-Learn library, as well as stopwords and punctuation libraries from the venerable NLTK (the Naural Language Took Kit).

In [5]:
def tokenize(text):
    # get the english stopwords
    nltk.download('stopwords', quiet=True)
    stopwords = nltk.corpus.stopwords.words('english')
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    nltk.download('punkt', quiet=True)
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.wordpunct_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        # include only those that contains letters
        if re.search('[a-zA-Z]', token):
            # exclude stop words, those shorter than 3 characters, and those that
            # start with non-alphanumeric characters
            if token not in stopwords and len(token) > 2 and token[0].isalnum():
                filtered_tokens.append(token)
    return filtered_tokens

### Feature Extraction

We will use a vectorizer called TF-IDF (Term-Frequency times Inverse Document-Frequency).

The tokenization method above converts extracted text into a feature array. We need this to feed for our algorithm that determines salient features by comparing the frequencies of these features across different documents.

Read more here: http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re

# Note: We have limited the max_features to 100000 here for faster processing
vectorizer = TfidfVectorizer(
    max_features=100000,
    stop_words='english',
    use_idf=True,
    tokenizer=tokenize)

feature_matrix = vectorizer.fit_transform(df['keywords'])
feature_names = vectorizer.get_feature_names()

### Clustering

For a simple demonstration, we use one of the fastest clustering algorithms available -- namely, k-means clustering.

In [7]:
from sklearn.cluster import KMeans

# Execute the k-means clustering
nclusters = 3 # Split into 3 clusters
km = KMeans(n_clusters=nclusters)
km.fit(feature_matrix)

# Create a dataframe out of the formed clusters
clusters = km.labels_.tolist()
docs = { 'id': df['id'], 'cluster': clusters }
clusters_df = pd.DataFrame(docs, columns = ['id', 'cluster'])

# Show a sample of the result of the clustering
clusters_df.sample(10)

,id,cluster
2885,08dcf00b-d07f-4763-aa37-d3dbef0fb763,0
4077,53615bb3-0f0c-438d-acf7-3a34aa010930,0
6168,974c3f38-9347-41a1-b4de-e7279a5fa11c,0
3134,c94430cc-5e16-44f3-b144-be90a9bd3dd5,0
3381,2c0a93ff-4699-4308-a776-fa68fa6a795b,0
2898,17b0382b-7fe3-415c-bcd8-2b8d8ff30d70,0
6752,2c2e0883-3ff3-4c68-9256-86437be21ecd,0
6789,6f0003b0-76d2-48e2-8d37-d8b3e5dd729b,0
3375,b3f9bdbd-f262-44aa-a63e-7e23508f3f0b,0
5026,26624e55-94e8-4974-ad51-b0f22c7e8244,0


To get the keywords/terms per cluster, we need to extract the k-means clustering object from the indices of terms that compose each cluster. In order to get the top defining terms, we reverse the ordering so we get the closest term to the cluster center as the first element in the list. These steps are accomplished in a single of code below.

In [8]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

In [9]:
# We proceed further to get only the top 20 terms per cluster
cluster_terms = {k: [] for k in clusters}
for i in range(nclusters):
    cluster_top_terms = [feature_names[x] for x in order_centroids[i, :20]]
    cluster_terms[i] = cluster_top_terms

df.reset_index(inplace=True)
clusters_df.reset_index(inplace=True)

# We fuse the raw dataframe with the clusters dataframe so get
# a mapping of the record ID, keywords, and cluster in one dataframe
df.set_index('id', inplace=True)
clusters_df.set_index('id', inplace=True)
#combined_df = df.join(clusters_df, how='outer', on='id')
combined_df = df.merge(clusters_df)

# Then from the combined dataframe we can generate a good summary
# of the clusters with cluster number, the count of records, and their defining terms
cdf = combined_df.groupby('cluster').agg('count')
cdf.rename(columns={'keywords': 'count'}, inplace=True)
cdf['keywords'] = [', '.join(cluster_terms[x]) for x in cdf.index]

for i,row in cdf.iterrows():
    print('Cluster #%s' % row.name)
    print(row.keywords, '\n')

Cluster #0
related, contact, information, september, categories, navigation, partners, newsletter, description, service, products, upcoming, archives, judonotes, twitter, education, similar, coupons, coupon, welcome 

Cluster #1
yachting, angling, telephone, galleries, fishing, private, competition, functions, sponsors, information, corporate, welcome, heaviest, weddings, history, inhouse, management, membership, monthly, committee 

Cluster #2
easy, nameclassifier, udt, author, status, phase, class, proposed, advanced, notes, project, modified, connection, appears, version, type, direction, generalization, complexity, priority 



We now have three clusters defined by a collection of characteristic keywords. The keywords associated with these clusters can be used to demonstrate record auto-classification.

### Saving the Clustering Results

For the auto-classification step, at least the generated clusters and the vectorizer object are needed. So we need to dumpt those into file. In Python, we do that using the `pickle` package. It serializes any complex Python object/data structure that can then be loaded later on for reuse.

In [10]:
import pickle
import os

results = {
    'vectorizer': vectorizer,
    'features': feature_matrix,
    'clusters': combined_df,
    'terms': cluster_terms
}

if not os.path.exists('temp'):
    os.mkdir('temp')
    
pickle.dump(results, open('temp/clusters.pkl', 'wb'))

The above step (saving the cluster model to disk) is necessary for the next stage, building an auto-classifier based on keywords. 